In [ ]:
# Variable set-up

rq = 'rq2' # Options: 'rq1', 'rq2'
cv = 'ss' # Options: 'ts' (time series split, ignore siblings), 'ss' (stratified shuffle, ignore siblings)
data_type = 'str' # Options: 'str' (just structured data), 'all' (structured data and list of strings)
algorithm_names = ['decision_tree', 'logistic_regression', 'gradient_boosting'] 
resampling_name = 'ada' # anything other than 'ada' does 'smote' 
select_features_alpha = 0.001 # Should be highest 0.001 as otherwise all dropped
rcv_n_iter = 50 # The more iterations, the more the randomised search searches for an optimal solution
parameters = 2 # 

# Don't change
file_stub_y_siblings = rq + '_' + cv + '_str' # use 'str' for all 
file_stub = rq + '_' + cv + '_' + data_type # Creates file stub for saving in the format e.g. rq1_ss_str
levers = resampling_name + '_' + str(select_features_alpha) + '_' + str(rcv_n_iter) + '_' + str(parameters)
print(file_stub + '_' + levers)

In [ ]:
# Additional variables

bootstrap_num = 500
algorithm_names = ['decision_tree', 'logistic_regression', 'gradient_boosting']


In [ ]:
## File directories
local_dir = '/Users/[username]/Documents/Final transfer out data and code to WWC Jan 2020' # insert [username]
hard_drive_dir = '/Volumes/diskAshur2/Final transfer out data and code to WWC Jan 2020/Data for model/Use'


### Below here - generic set-up - don't vary. Change which models are run and output saved by varying the parameters above


In [ ]:
# Load user-written functions 

get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')

import analysis_functions

In [ ]:
# Set working directory
import os
import pickle
os.chdir(hard_drive_dir)

In [ ]:
# Import structured data -train, test split

# Import structured data -train, test data
import pandas as pd
X_tr = pd.read_csv("Final/X_train_{}_final.csv".format(file_stub), index_col = 0)
print(X_tr.shape)
X_tr.reset_index(inplace = True, drop = True)
print(X_tr.index)

X_test = pd.read_csv("Final/X_test_{}_final.csv".format(file_stub), index_col = 0)
print(X_test.shape)
X_test.reset_index(inplace = True, drop = True)
print(X_test.index)

y_tr = pd.read_csv("y_train_{}.csv".format(file_stub_y_siblings), index_col = 0, header = None)
print(y_tr.shape)
y_tr.reset_index(inplace = True, drop = True)
y_tr = pd.Series(y_tr[1])
print(y_tr.index)

y_test = pd.read_csv("y_test_{}.csv".format(file_stub_y_siblings), index_col = 0, header = None)
print(y_test.shape)
y_test.reset_index(inplace = True, drop = True)
y_test = pd.Series(y_test[1])
print(y_test.index)

siblings_tr = pd.read_csv("siblings_train_{}.csv".format(file_stub_y_siblings), index_col = 0, header = None)
print(siblings_tr.shape)
siblings_tr.reset_index(inplace = True, drop = True)
siblings_tr = pd.Series(siblings_tr[1])
print(siblings_tr.index)

siblings_test = pd.read_csv("siblings_test_{}.csv".format(file_stub_y_siblings), index_col = 0, header = None)
print(siblings_test.shape)
siblings_test.reset_index(inplace = True, drop = True)
siblings_test = pd.Series(siblings_test[1])
print(siblings_test.index)

In [ ]:
# Drop key columns (after data is saved as still need the keys for merging in 7a_Combine_and_Split_Data and 8_Fairness)
print(X_tr.shape)
print(X_test.shape)

key_cols = ['PSID',  'ReferralDatetime', 'ReferralDatetime_month_year']
X_tr.drop(columns = key_cols, inplace = True, errors = 'ignore')
X_test.drop(columns = key_cols, inplace = True, errors = 'ignore')
print(X_tr.shape)
print(X_test.shape)

X_tr = X_tr.select_dtypes(include = 'number') 
X_test = X_test.select_dtypes(include = 'number') 
print(X_tr.shape)
print(X_test.shape)

In [ ]:
# Import best model
import numpy as np
import pandas as pd

model_outputs = []
try:
    model_output_dtc = pd.read_csv("{}/Models/model_output_{}_decision_tree_{}.csv".format(local_dir, file_stub, levers), index_col = 0)
    model_outputs.append(model_output_dtc)
except(FileNotFoundError):    
    pass

try:
    model_output_lr = pd.read_csv("{}/Models/model_output_{}_logistic_regression_{}.csv".format(local_dir, file_stub, levers), index_col = 0)
    model_outputs.append(model_output_lr)
except(FileNotFoundError):    
    pass

try:
    model_output_gbc = pd.read_csv("{}/Models/model_output_{}_gradient_boosting_{}.csv".format(local_dir, file_stub, levers), index_col = 0)
    model_outputs.append(model_output_gbc)
except(FileNotFoundError):    
    pass

model_output = pd.concat(model_outputs, axis = 0, ignore_index = True)
# Find the model which finds the best worst case scenario
max_min_test_score = np.argmax(model_output['mean_test_score'] - 2* model_output['std_test_score']) 
best_algorithm = model_output.loc[max_min_test_score,'algorithm']
print("Best algorithm: ", best_algorithm)
print("Max min score", model_output.loc[max_min_test_score, 'mean_test_score'])

In [ ]:
import pickle

# Load saved models
filename = open("{}/Models/best_estimator_{}_dict.pkl".format(local_dir, file_stub), "rb")
best_estimator_dict = pickle.load(filename)
fitted_model = best_estimator_dict[best_algorithm]


In [ ]:
## Predictive Inference

# Bootstrapping for prediction intervals
# https://stackoverflow.com/questions/47414842/confidence-interval-of-probability-prediction-from-logistic-regression-statsmode

preprocessor = fitted_model['preprocessor']
preds = {k: [] for k in X_test.index}
for i in range(bootstrap_num):
    print('Bootstrap number: ', i)
    boot_idx = np.random.choice(X_tr.index, replace=True, size=X_tr.shape[0]) # pick indices at random of length of X_tr
    if data_type == 'all':
        X_tr_transformed = pd.DataFrame(preprocessor.fit_transform(X_tr))
        fitted_model.fit(X_tr.loc[boot_idx,], y_tr[boot_idx]) # Fit the model
    else:
        try:
            fitted_model.fit(X_tr.loc[boot_idx,], y_tr[boot_idx]) # Fit the model
        except(ValueError):
            continue
    y_pred_prob_1 = [prob[1] for prob in fitted_model.predict_proba(X_test)] # Find the probabilities for all the predictions
    y_pred_prob_1 = pd.Series(y_pred_prob_1, index = X_test.index) # Add the index
    for idx in y_pred_prob_1.index:
        preds[idx].append(y_pred_prob_1[idx]) # for each index, append the probability to a list


In [ ]:
# Individual predictions (90% prediction intervals)
# Average width of prediction intervals
import numpy as np
widths, mean_probabilities = [], []
for key, values in preds.items():
    width = 1.645 * (np.std(preds[key]) / np.sqrt(len(preds[key]))) * 2 # 1.645 is z value for 90% interval
    widths.append(width)
    mean_probability = round(np.mean(preds[key]),2) # take the mean of the prediction probabilities
    mean_probabilities.append(mean_probability)

preds_interval_widths = dict(zip(preds.keys(), widths))
preds_interval_mean_probs = dict(zip(preds.keys(), mean_probabilities))
average_width = round(np.mean(widths), 4)
print("Averge width of prediction interval: ", average_width)

In [ ]:
# Save the bootstrapped predictions 

with open("{}/Models/Prediction Intervals/preds_interval_widths_{}.pkl".format(local_dir, file_stub), "wb") as handle:
    pickle.dump(preds_interval_widths, handle, protocol = pickle.HIGHEST_PROTOCOL)
    
with open("{}/Models/Prediction Intervals/preds_interval_mean_probs_{}.pkl".format(local_dir, file_stub), "wb") as handle:
    pickle.dump(preds_interval_mean_probs, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [ ]:
# Histogram of predicted probabilities
get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt

algorithm_names_full = ['Decision tree', 'Logistic regression', 'Gradient boosting']

if cv == 'ts':
    cv_for_graph = 'predicting the future'
if cv == 'ss':
    cv_for_graph = 'predicting contemporaneously'


all_probabilities = [item for sublist in preds.values() for item in sublist]

p = np.array(all_probabilities)
plt.hist(p, density = False, color = '#ff7057')
plt.xlabel('Predicted Probabilities')
plt.ylabel('Frequency')
plt.title('Histogram of Predicted Probabilities {}'.format(cv_for_graph))
plt.savefig('{}/Graphs/Histogram of Predicted Probabilities {} ({}).png'.format(local_dir, file_stub, cv_for_graph), transparent=False, dpi=80, bbox_inches="tight")
plt.show()

In [ ]:
print(X_tr.shape)
print(X_test.shape)

In [ ]:
# Check different thresholds
# high threshold for positive class means model is more precise
# low threshold for positive class means model recalls more
from sklearn.metrics import precision_score, recall_score, fbeta_score

spacing = 0.03
thresholds = np.arange(0.1,1,0.1)
thresholds = [round(t, 2) for t in thresholds]

precision_scores, recall_scores, widths_at_interval = [], [], []
f_scores = {}
y_pred_proba = fitted_model.predict_proba(X_test)
for threshold in thresholds:
    y_test_predictions = y_pred_proba[:,1] > threshold
    precision = round(precision_score(y_test, y_test_predictions), 2)
    recall = round(recall_score(y_test, y_test_predictions), 2)
    f_score = round(fbeta_score(y_test, y_test_predictions, beta = 0.1), 2)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f_scores[threshold]= f_score
    print("Precision (threshold = {}): ".format(threshold), precision)
    print("Recall (threshold = {}): ".format(threshold), recall)
    print("F score (threshold = {}): ".format(threshold), f_score)
    # Prediction interval of the threshold value 
    indices_threshold = [k for k, v in preds_interval_mean_probs.items() if (v >= threshold - spacing) & (v <= threshold + spacing)]
    widths_threshold = [v for k, v in preds_interval_widths.items() if k in indices_threshold]
    width_at_threshold = round(np.mean(widths_threshold), 4)
    print("Width of prediction interval at threshold value: ", width_at_threshold)
    widths_at_interval.append(width_at_threshold)
    

In [ ]:
# Save results
predictions_intervals_all_df = pd.DataFrame({'Threshold': thresholds, 'Precision': precision_scores, 
                                             'Recall': recall_scores, 'F score (beta = 0.1)': f_scores.values(),
                          'Prediction interval (threshold +/- {})'.format(spacing): widths_at_interval})
predictions_intervals_all_df.to_csv('{}/Models/Prediction Intervals/Prediction intervals - all {}.csv'.format(local_dir, file_stub))


In [ ]:
# Prediction interval of the threshold value
threshold = max(f_scores, key=f_scores.get) 
print('Threshold which f-score is highest: ', threshold)
indices_threshold = [k for k, v in preds_interval_mean_probs.items() if (v >= threshold - spacing) & (v <= threshold + spacing)]
widths_threshold = [v for k, v in preds_interval_widths.items() if k in indices_threshold]
width_at_threshold = round(np.mean(widths_threshold), 4)
print("Width of prediction interval at threshold value: ", width_at_threshold)

In [ ]:
# Save results

prediction_intervals_df = pd.DataFrame({'Average width of prediction interval': [average_width], 'Width of prediction interval at threshold value': [width_at_threshold]})
prediction_intervals_df.to_csv('{}/Models/Prediction Intervals/Prediction intervals - max, ave{}.csv'.format(local_dir, file_stub))
